

# CrossData



**Documentación online** sobre Crossdata:

https://platform.bbva.com/en-us/developers/sandbox/documentation/tutorials-how-to-guides/sandbox-user-guide/using-crossdata



En el siguiente documento se explica con detalle el Uso de CrossData en Sandbox con la herramienta Microstrategy.

Documentación "**Guia utilización CrossData desde MicroStrategy**":

https://docs.google.com/document/d/1wRxuVg9zlVUT4bKcZKuCZ1TNMyrDgNjaGKvujGtlxfg/edit



## Creación de XDSession

XDSession extiende a SparkSession.
Todas las funcionalidades de la SparkSession estan accesibles por tanto a traves de la XDSession.

In [0]:
from pystratio.xd.xdsession import XDSession

sandbox = "fper" # here goes the name of your sandbox

with open("/tmp/custom_storage_crossdata.conf","w") as conffile:
    conffile.write("crossdata.storage.persistence=\"parquet\"\n")
    conffile.write("crossdata.storage.path=\"/data/sandboxes/"+sandbox+"/data/tmp\"\n")
    
xd = XDSession(sc,configFile="/tmp/custom_storage_crossdata.conf")

ModuleNotFoundError: No module named 'pystratio'



- Si imprimimos la variable **xd** vemos que tiene **CrossData Session**. 
- Si imprimimos la variable **spark**, vemos que contiene la **SparkSession**.
- La variable xd la hemos asignado nosotros a mano. La variable spark se asigna al iniciar el kernel de pyspark en Datio. Sería lo mismo que hacer: 
    
    *from pyspark.sql import SparkSession*
    
    *spark = SparkSession.builder.getOrCreate()*

In [0]:
print(xd)
print(spark)

NameError: name 'xd' is not defined



En Datio el catálogo de **spark** (el metastore) **no es persistente** y cuando reiniciamos el kernel desaparece. Los otros usuarios del Sandbox no pueden ver las tablas/bases de datos creadas con la SparkSession.

El catálogo de **Crossdata** sí es **persistente**, y los todos los usuarios del Sandbox pueden ver las tablas/bases de datos creadas por los demás.



### Diferencia SparkSession y XDSession

**Show Databases**  

*(Por defecto la unica base de datos existente es la base de datos **"default"**)*  



Creamos una nueva Base de datos con Spark

In [0]:
spark.sql("create database if not exists db_spark")

DataFrame[]



Mostramos las Bases de datos existentes en el catálogo de Spark

In [0]:
spark.sql("show databases").toPandas()

,databaseName
0,bbdd_spark
1,default


In [0]:
spark.sql("show tables").toPandas()

,database,tableName,isTemporary




Ahora mostramos las bases de datos existentes en el catálogo de CrossData

In [0]:
xd.sql("create database if not exists db_xd")

DataFrame[]

In [0]:
xd.sql("show databases").toPandas()

,databaseName
0,bbddXD
1,bbdd_xd
2,bbdd_XD
3,bbdd_spark
4,nomeacuerdoxd
5,SYNERGICV
6,football_events
7,default
8,pruebaXD
9,SYNERGIC




Como vemos ambos catálogos difieren, incluso la base de datos por defecto _"default"_ es una base de datos diferente en cada uno de los catalogos.



## Catalogo de Crossdata




Creamos una base de datos en Crossdata. Ya lo hemos hecho arriba, pero recordamos nuevamente como creamos una base de datos.

In [0]:
xd.sql("create database if not exists db_xd")

DataFrame[]



Volvemos a mostrar las bases de datos existentes en el catálogo de CrossData

In [0]:
xd.sql("show databases").toPandas()

,databaseName
0,bbddXD
1,bbdd_xd
2,bbdd_XD
3,bbdd_spark
4,nomeacuerdoxd
5,SYNERGICV
6,football_events
7,default
8,pruebaXD
9,SYNERGIC




Vamos a usar la base de datos recién creada

In [0]:
xd.sql("use db_xd")

NameError: name 'xd' is not defined



Vamos a crear un fichero parquet en HDFS.
Generaremos los datos dentro del propio notebook.

In [0]:
path_sb = "/data/sandboxes/"+sandbox+"/data/"

NameError: name 'sandbox' is not defined

In [0]:
from pyspark.sql import Row
testRow = Row("Id", "Name", "City")
rows = [testRow(1, "Edward", "Boston"), testRow(2, "Ariadna", "New York"), testRow(3, "Lewis", "Austin")]
df = xd.createDataFrame(rows)
df.write.mode("overwrite").parquet(path_sb+"tmp/names_and_cities")



### Creamos una tabla en el catálogo de XD a partir de un DF

In [0]:
df = xd.read.parquet(path_sb+"tmp/names_and_cities")

In [0]:
df.show()

+---+-------+--------+
| Id|   Name|    City|
+---+-------+--------+
|  2|Ariadna|New York|
|  3|  Lewis|  Austin|
|  1| Edward|  Boston|
+---+-------+--------+





#### Forma 1

In [0]:
df.registerTempTable("mytable")



La hacemos no temporal

In [0]:
xd.sql('create table if not exists persisted_table as select * from mytable')

DataFrame[]

In [0]:
xd.sql("show tables").show()

+-------------+----------------+-----------+
|     database|       tableName|isTemporary|
+-------------+----------------+-----------+
|nomeacuerdoxd|  mitabla_forma2|      false|
|nomeacuerdoxd| mitablapersiste|      false|
|nomeacuerdoxd|names_and_cities|      false|
|nomeacuerdoxd|        t_prueba|      false|
|             |         mitabla|       true|
+-------------+----------------+-----------+





#### Forma 2

In [0]:
df.write.mode('overwrite').saveAsTable('persisted_table_2')

In [0]:
xd.sql('show tables').show()

+-------------+----------------+-----------+
|     database|       tableName|isTemporary|
+-------------+----------------+-----------+
|nomeacuerdoxd|  mitabla_forma2|      false|
|nomeacuerdoxd| mitablapersiste|      false|
|nomeacuerdoxd|names_and_cities|      false|
|nomeacuerdoxd|        t_prueba|      false|
|             |         mitabla|       true|
+-------------+----------------+-----------+





### Creamos una tabla a partir de un fichero en HDFS



Los datos ya están almacenados en el HDFS y lo único que hacemos es dar de alta una tabla nueva en el catálogo de Crossdata que apunta al fichero parquet.

In [0]:
xd.sql("CREATE TABLE IF NOT EXISTS names_and_cities \
       USING parquet OPTIONS (path '{}')".format(path_sb+"tmp/names_and_cities"))

xd.sql("show tables").toPandas()

,database,tableName,isTemporary
0,nomeacuerdoxd,mitabla_forma2,False
1,nomeacuerdoxd,mitablapersiste,False
2,nomeacuerdoxd,names_and_cities,False
3,nomeacuerdoxd,t_prueba,False
4,,mitabla,True


In [0]:
xd.sql("SELECT * FROM names_and_cities").toPandas()

,Id,Name,City
0,2,Ariadna,New York
1,3,Lewis,Austin
2,1,Edward,Boston




Vemos que al leer con SQL un archivo parquet, guarda también el tipo de dato al que pertenecen las columnas.

In [0]:
xd.table('names_and_cities').dtypes

[('Id', 'bigint'), ('Name', 'string'), ('City', 'string')]



**Nota:**

Si sólo tenemos el archivo en CSV, mejor hacer: df = spark.read.csv(..., inferSchema=True, header=True.. )

Porque si lo hacemos como acabamos de ver con SQL y parquet, pero con CSV, perdemos el tipo de dato al que pertenecen. Aparecen todas como string.



Podemos hacer multiples consultas SQL con estos datos obtenidos

In [0]:
mydata = xd.sql("SELECT * FROM names_and_cities WHERE Id=2")
mydata.show()

+---+-------+--------+
| Id|   Name|    City|
+---+-------+--------+
|  2|Ariadna|New York|
+---+-------+--------+





* Guardamos el resultado con el metodo write

In [0]:
mydata.write.mode('overwrite').parquet(path_sb+'tmp/names_and_cities_id2') 



### Creamos una tabla y un parquet a partir de operaciones sobre otra tabla

In [0]:
xd.sql("create table if not exists names_and_cities_id1 as select * from names_and_cities where Id=1")

DataFrame[]

In [0]:
xd.sql("show tables").show()

+-------------+--------------------+-----------+
|     database|           tableName|isTemporary|
+-------------+--------------------+-----------+
|nomeacuerdoxd|      mitabla_forma2|      false|
|nomeacuerdoxd|     mitablapersiste|      false|
|nomeacuerdoxd|    names_and_cities|      false|
|nomeacuerdoxd|names_and_cities_id1|      false|
|nomeacuerdoxd|            t_prueba|      false|
|             |             mitabla|       true|
+-------------+--------------------+-----------+





En el diretorio por defecto que hemos establecido en Crossdata crea un parquet.

Actualmente, al hacer un drop borrará sólo la tabla en Crossdata y no el parquet. Debería borrar ambos, pero actualmente (enero 2019) no es así.

In [0]:
xd.sql('drop table names_and_cities_id1')

DataFrame[]

In [0]:
xd.sql("show tables").show()

+-------------+----------------+-----------+
|     database|       tableName|isTemporary|
+-------------+----------------+-----------+
|nomeacuerdoxd|  mitabla_forma2|      false|
|nomeacuerdoxd| mitablapersiste|      false|
|nomeacuerdoxd|names_and_cities|      false|
|nomeacuerdoxd|        t_prueba|      false|
|             |         mitabla|       true|
+-------------+----------------+-----------+





### Trabajando con SparkSession directamente ###

Ya hemos visto como trabajar con el catalogo de XD usando la XDSession. Ahora vamos a ver como recuperar informacion directamente usando Spark.

In [0]:
spark.sql("show databases").toPandas()

,databaseName
0,bbdd_spark
1,default


In [0]:
spark.sql("show tables").toPandas()

,database,tableName,isTemporary




Usamos el objeto `spark`, que se trata de la sesion de spark sin la funcionalidad de la XDSession.

Leemos el fichero directamente y creamos un DF

In [0]:
df_spark = spark.read.parquet(path_sb+"tmp/names_and_cities")

In [0]:
df_spark.show(5)

+---+-------+--------+
| Id|   Name|    City|
+---+-------+--------+
|  2|Ariadna|New York|
|  3|  Lewis|  Austin|
|  1| Edward|  Boston|
+---+-------+--------+





Podemos guardar la tabla como una tabla temporal, o como una tabla no temporal. 

Si la creamos como **temporal** no pertenecerá a ninguna base de datos y no persistirá en el tiempo.

Si la creamos **no temporal**, pertenecerá a una base de datos, pero tampoco persistirá en el tiempo, pues estamos usando Spark en lugar de Crossdata. Si fuera con Crossdata, sí persistiría.

In [0]:
df_spark.registerTempTable("names_cities_tmp")

spark.sql("show tables").toPandas()

,database,tableName,isTemporary
0,,names_cities_tmp,True




Con registerTempTable registramos una tabla temporal en la Spark Session. No tendrá una base de datos definida.

Con saveAsTable, registramos una tabla no temporal, pero aún así al hacerlo con la Spark Session no será persistente.

In [0]:
df_spark.write.mode('overwrite').saveAsTable('names_cities_persist')

spark.sql("show tables").toPandas()

,database,tableName,isTemporary
0,default,names_cities_persist,False
1,,names_cities_tmp,True




Comprobamos el contenido

In [0]:
spark.sql("SELECT * FROM names_cities_persist").toPandas()

,Id,Name,City
0,2,Ariadna,New York
1,3,Lewis,Austin
2,1,Edward,Boston
